In [1]:
import os
import rasterio
from rasterio import windows
from rasterio.enums import Resampling
from scipy.ndimage import generic_filter
import pandas as pd
import numpy as np
import re
from datetime import datetime
print("modules imported")

modules imported


In [2]:
dem = r"D:/ASOML/Sierras/features/ASOML_SNM_dem_60_albn83.tif"

Functions all below

In [3]:
def slope_std_3x3(slope_raster_path, output_path):
    with rasterio.open(slope_raster_path) as src:
        slope = src.read(1, masked=True)
        profile = src.profile.copy()

    # Replace NoData with np.nan for proper std calculation
    slope_data = slope.filled(np.nan)

    # Define function that computes std ignoring NaNs
    def nanstd_filter(x):
        return np.nanstd(x)

    # Apply 3x3 window
    slope_std = generic_filter(
        slope_data,
        function=nanstd_filter,
        size=3,
        mode='nearest'
    )

    # Restore NoData areas
    slope_std = np.where(np.isnan(slope_data), -9999, slope_std)

    # Save to output raster
    profile.update(dtype='float32', nodata=-9999)

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(slope_std.astype('float32'), 1)

In [4]:
def elevation_std_3x3(dem_path, output_path):
    with rasterio.open(dem_path) as src:
        dem = src.read(1, masked=True)
        profile = src.profile.copy()

    # Replace masked/no-data values with np.nan
    dem_data = dem.filled(np.nan)

    # Define a std function that ignores NaNs
    def nanstd_filter(x):
        return np.nanstd(x)

    # Apply 3x3 moving window standard deviation
    elevation_std = generic_filter(
        dem_data,
        function=nanstd_filter,
        size=3,
        mode='nearest'
    )

    # Set NaNs back to a NoData value (e.g., -9999)
    elevation_std = np.where(np.isnan(dem_data), -9999, elevation_std)

    # Update metadata and save
    profile.update(dtype='float32', nodata=-9999)

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(elevation_std.astype('float32'), 1)

In [5]:
# this is to great the gradient mangintue 
def compute_gradient_magnitude(dem_path, output_path):
    with rasterio.open(dem_path) as src:
        dem = src.read(1, masked=True)
        transform = src.transform
        res_x = transform[0]    # pixel width
        res_y = -transform[4]
        profile = src.profile.copy()

    # Compute partial derivatives using numpy gradient
    dem_data = dem.filled(np.nan)
    dz_dy, dz_dx = np.gradient(dem, res_y, res_x)  # note: y comes first
    grad_mag = np.sqrt(dz_dx**2 + dz_dy**2)
    grad_mag = np.where(np.isnan(dem_data), -9999, grad_mag)

    # Save the result as a new raster
    profile = src.profile.copy()
    profile.update(dtype='float32', nodata=None)

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(grad_mag.astype('float32'), 1)

In [ ]:
compute_gradient_magnitude(dem, r"D:/ASOML/Sierras/features/ASOML_SNM_gradMag_60_albn83.tif")

In [ ]:
elevation_std_3x3(dem, r"D:/ASOML/Sierras/features/ASOML_SNM_stdElv_60_albn83.tif")

C:\Users\etyrr\anaconda3\envs\aso-dl\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
slope_std_3x3(r"D:\ASOML\Sierras\features\ASOML_SNM_slope_60_albn83.tif", r"D:\ASOML\Sierras\features\ASOML_SNM_STDslope_60_albn83.tif")